<a href="https://colab.research.google.com/github/18708064/big-data/blob/master/postblock2_q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Map function
def map(filename, content):
    # For each character in the content of the file
    for char in content:
        # Emit the character as the key and the value as 1
        EmitIntermediate(char, 1)

# Reduce function
def reduce(char, values):
    # Sum the counts for each character key
    result = sum(values)
    Emit(char, result)


Question 3.5

In [ ]:
# Combiner function
def combiner(char, values):
    # Partially sum values before sending to reducer
    partial_sum = sum(values)
    Emit(char, partial_sum)


Question 4

In [1]:
# Map function
def map(hostname, document):
    # Tokenize the document into words
    words = tokenize(document)
    term_vector = {}

    # Count word occurrences
    for word in words:
        if word in term_vector:
            term_vector[word] += 1
        else:
            term_vector[word] = 1

    # Emit hostname and term vector
    EmitIntermediate(hostname, term_vector)

# Reduce function
def reduce(hostname, term_vectors):
    final_term_vector = {}

    # Aggregate term vectors across all documents for the hostname
    for term_vector in term_vectors:
        for word, count in term_vector.items():
            if word in final_term_vector:
                final_term_vector[word] += count
            else:
                final_term_vector[word] = count

    # Emit hostname and final term vector
    Emit(hostname, final_term_vector)
